<a href="https://www.kaggle.com/code/robertmorsch/space-titanic-eda-cv-hyperopt-automl?scriptVersionId=122037845" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

Hello and welcome to my Space Titanic effort to predict which passengers are transported to an alternate dimension.
This is my analysis of the Kaggle dataset "spaceship-titanic"

NOTE FOR BEGINNERS: this notebook is merely the final notebook of my analytic work for this competition. I recommend, if you are a beginner, to approach your work similar to writing a draft in school. Begin by dumping your thoughts outs, and iterate by addings structure and removing junk. Don't let yourself get stuck trying to write eloquently, automating, cleaning, or engineering features, on your first go around.

In [ ]:
#Importing packages
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
#Pulling in the data
submission = pd.read_csv("/kaggle/input/spaceship-titanic/sample_submission.csv")
passengers = pd.read_csv("/kaggle/input/spaceship-titanic/train.csv")
testset_passengers = pd.read_csv('/kaggle/input/spaceship-titanic/test.csv')

# Exploring the dataset as a whole

In [ ]:
#Getting to know the data.
#Its rows vs columns
print("Shape: ",passengers.shape)
print("\n")
#Rows containing missing data by column
print("Nulls by Col: ","\n")
print(passengers.isnull().sum().sort_values())
print("\n")
#Total missing data points
print("Total Nulls: ",passengers.isnull().sum().sum())

Decent amount of rows, far from big data. 14 Columns is fairly rich and give some decent options, however Transported is the TARGET variable, and PassengerID will pointless due to its high cardinality.

In [ ]:
#Columns and their datatype
#DataFrame 5 row sample
passengers.info()
passengers.head()

In previous interations, I explored and confirmed the following assumptions. But from what we can see from the sample, the attention grabbing features are that we can transform upfront.
* True/False columns that should be transformed into 1/0: CryoSleep, VIP, and Transported. 
* Cabin is a compound column that should be split into 3 separate columns: deck, num, and side.
* Name into 2 columns: first_name and last_name.
    * The primary reason for this is that we can create and easy new column from this by calculating family size aboard. 
    * Names are rarely useful in themselves due to their high cardinality.

This function is meant to apply the changes I identified in the previous text block. The reason it is a function is so I can reuse it for the TEST DATASET. This is the reason for the IF, the TEST DATASET does not contain the Transported column.

In [ ]:
def earlyTransformations(dataset, is_training_set = True):
    #Will likely not be used - Dropping PassengerID
    if is_training_set:
        dataset[['Transported']] = dataset[['Transported']]*1
    
    #Transforming True/False to 1/0
    dataset[['CryoSleep','VIP']] = dataset[['CryoSleep','VIP']]*1
    
    #Splitting Cabin into deck/num/side columns
    #Dropping the Cabin column
    dataset[['deck','num','side']] = dataset['Cabin'].str.split(pat='/',expand=True)
    dataset = dataset.drop('Cabin',axis=1)
    dataset['num'] = dataset['num'].astype('float64')

    #Spliting Name into First Name and Last Name
    #Creating Family Size column from number of occurences of a passenger's last name
    dataset[['first_name', 'last_name']] = dataset['Name'].str.split(expand=True)
    dataset['family_size'] = dataset.groupby('last_name')['last_name'].transform('count')
    
    return dataset

Observing the changes made to the dataset

In [ ]:
passengers = earlyTransformations(passengers)
passengers.info()
passengers.head()

Checking the missing data per column. This is an extremely important even if it may appear to be redundant. Unfortunately, manipulating columns can often result in, missing data no longer registering as missing. Although, I am not entirely sure of the implications of missing data as not registering as missing, but it is best likely best avoided.

In [ ]:
passengers.isnull().sum().sort_values()

# Exploratory Data Analysis

Exploring the data, starting with categorical features. Primarily, we are trying to get to know the data for key features or actionable engineering. For example most of the tranformations performed earlier in the notebook were discovered in a previous iteration during EDA.

# Exploring Categorical Features

Following code is an iterative function to output visualzations with 2 columns.
* LEFT column: show the distribution across the categories in each column. 
* RIGHT column: split of the column categories by TARGET value.

In [ ]:
# Function to display distribution of Categorical columns
# Will display 2 visuals per column
# Will need to have columns chosen before hand.
def cat_cols_display(df):
    
    plt.figure(figsize=(15, 20))
    plt.subplots_adjust(hspace=0.5)
    plt.suptitle("Cat Feats", fontsize=18, y=0.95)

    cat_features = df.columns
    number_of_columns = len(cat_features)
    n=1
    
    # loop through the length of tickers and keep track of index
    for cat in cat_features:
        # add a new subplot iteratively
        ax = plt.subplot(number_of_columns, 2,n)

        # filter df and plot ticker on the new subplot axis
        sns.countplot(x=passengers[cat],
                     order = passengers[cat].value_counts().index)

        # chart formatting
        ax.set_title(cat.upper())
        #ax.get_legend().remove()
        ax.set_xlabel("")
        ax.bar_label(ax.containers[0])

        n +=1 #It

        ax2 = plt.subplot(number_of_columns, 2,n)
        sns.countplot(x=passengers[cat],
                  order = passengers[cat].value_counts().index,
                 hue =passengers['Transported'])

        n +=1

        # chart formatting
        ax2.set_title(cat.upper())
        #ax.get_legend().remove()
        ax2.set_xlabel("")
        ax2.bar_label(ax2.containers[0])

In [ ]:
#Checking which columns are categorical
#Calling the function to display exploratory visualizations of categorical columns
print(passengers.select_dtypes(include='object').columns)
cat_cols_display(passengers[['HomePlanet', 'CryoSleep', 'Destination', 'VIP', 'deck', 'side']])

Notes on categorical visualizations
* There are a few columns with sizeable discrpencies. Discrepencies being imbalance in TARGET value in a category within a column.
* CyroSleep, HomePlanet, and deck having the largest discrepencies, so we should expect these to be the most important for predicting Transported.

# Exploring Continuous Features

Similar visualization function as the one for categorical features.
Iterative function to output visualzations with 2 columns.
* LEFT column: distribution of numbers contained in the feature, organized into bins. 
* RIGHT column: violin plot, split between non transported vs transported.

In [ ]:
# Function to display distribution of Continuous columns
# Will display 2 visuals per column
# Will need to have columns chosen before hand.

def continuous_cols_display(df):
    
    plt.figure(figsize=(15, 40))
    plt.subplots_adjust(hspace=0.5)
    plt.suptitle("Continuous Feats", fontsize=18, y=0.95)

    continuous_features = df.columns
    subplot_row = len(continuous_features)
    n=1
    
    # loop through the length of tickers and keep track of index
    for feat in continuous_features:
        # add a new subplot iteratively
        ax = plt.subplot(subplot_row, 2,n)

        # filter df and plot ticker on the new subplot axis
        sns.histplot(data=passengers,
                     x=feat,
                    bins=50)

        # chart formatting
        ax.set_title(feat.upper())
        #ax.get_legend().remove()
        ax.set_xlabel("")
        n +=1 #Iterate subplot location

        ax2 = plt.subplot(subplot_row, 2,n)
        sns.violinplot(data=passengers, 
                       x="Transported", 
                       y=feat)

        n +=1 #Iterate subplot location

        # chart formatting
        ax2.set_title(feat.upper())
        ax2.set_xlabel("Transported")

Quick view of the descriptive statistics of the coninuous features.

In [ ]:
passengers.describe()

In [ ]:
print(passengers.select_dtypes(exclude='object').columns)
df = passengers[['Age', 'RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck', 'num', 'family_size']]
continuous_cols_display(df)

Notes on categorical visualizations
* the spending features: RoomService, FoodCourt, ShoppingMall, Spa, and VRDeck have huge skews in their distributions between the target feature.
* Due to their similar nature (spending money), I will be adding 2 columns to combine these columns: Spending_Total and a binary Spent_Money.
* The other distributions have some discrepency but not nearly as much. 

In conclusion, the violin plots for the most part are almost mirror images, except for the spending features, which have some large discrepencies. Most notably: VRDeck, Spa, and RoomService.

In [ ]:
corr_matrix=passengers.corr()
corr_matrix['Transported'].sort_values(ascending=False)

Correlation confirms the large discrepencies we saw in the violin plots, where VRDeck, Spa, and RoomService have the strongest relationship.

# Data Processing Pipeline

We are now ready to clean the data. I am doing this through a pipeline. Creating a pipeline is not necessary, but will allow the following code block to be used in future analyses.
Quick notes on whats happening in pipeline:
* There will be a split in inputs, continuous features and categorical features,leading to a combined output at the end of the pipeline.
* SimpleImputer is meant to replace missing values. Here I am using the median for continuous and catch-all 'missing' for categorical.
* StandardScaler to normalize the continuous features to have the same mean and standard deviation. Allowing for the analysis of continuous features on the same scale.
* OneHotEncoder generates a new column for each category in every categrical coloumn. Otherwise, the categorical columns are inefficient or impossible to analysis in most ML algorithms.

In [ ]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler

#Numeric Pipleine Steps
numerical_imputer = SimpleImputer(strategy='median')
numerical_scaler = StandardScaler()
    
#Categorical Pipleine Steps   
categorical_imputer = SimpleImputer(strategy='constant',
                                    fill_value='missing')
categorical_missing_encoder = OneHotEncoder(handle_unknown='ignore')

Within the pipeline I will be adding 2 new continous columns using the class below.
* spent_total is the sum of all money spending columns
* spent_money is binary, 1 for if spent_total is > 0 and 0 for if spent_total is <=0 

In [ ]:
#Class to create 2 new columns step in pipeline

#Dynamically finding the columns needed in the 
col_names = 'RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck'
RoomService_ix, FoodCourt_ix, ShoppingMall_ix, Spa_ix, VRDeck_ix = [
    passengers.columns.get_loc(c) for c in col_names] # get the column indices

from sklearn.base import BaseEstimator, TransformerMixin

class create_columns_for_spending(BaseEstimator, TransformerMixin):
    '''select specific columns of a given dataset'''
    def __init__(self):
        self
        
    def fit(self, X, y=None):
        return self
    
    def transform(self, X, y=None):
        spent_total = X[:, RoomService_ix] + X[:, FoodCourt_ix] +  X[:, ShoppingMall_ix] + X[:, Spa_ix] +  X[:, VRDeck_ix]
        spent_money = np.where(spent_total > 0, 1, 0)       
        return np.c_[X, spent_total, spent_money]

#attr_adder = create_columns_for_spending()
#passengers_extra_attribs = attr_adder.transform(passengers.values)

Quick view of the info and head of the dataset before inset into the pipeline

In [ ]:
passengers.info()
passengers.head()

Identifying the continuous and categorical features. Creating continuous pipeline and categorical pipeline. Then combining these two pipelines into the full pipeline.

In [ ]:
#Pipeline builds

#Identify and split the column names into numerical or categorical columns
numerical_features = ['Age', 'RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck'
                      ,'num', 'family_size', 'CryoSleep', 'VIP']
categorical_features = ['HomePlanet', 'Destination', 'deck', 'side']


numerical_pipeline = Pipeline([
    ('num_imputer',numerical_imputer),
    ('scaler',numerical_scaler)
])

categorical_pipeline = Pipeline([
    ('cat_imputer',categorical_imputer),
    ('categorical_missing_encoder',categorical_missing_encoder)
])
     
full_pipeline = ColumnTransformer([
    ('num',numerical_pipeline,numerical_features),
    ('cat',categorical_pipeline,categorical_features)
])

In [ ]:
passengers_labels = passengers['Transported'].copy()
passengers_prepared = full_pipeline.fit_transform(passengers)
passengers_prepared

Output of the pipeline is a numpy array

# Cross Validation and Modeling

Now the fun part. We are ready to train and validate our models.

Function to run basic cv predictions, to get a baseline comparison of the models.

In [ ]:
from sklearn.model_selection import cross_val_score

def CrossValidate_Model(model,X,y):
    # Multiply by -1 since sklearn calculates *negative* MAE
    scores = cross_val_score(model, X, y,
                                  cv=5,
                                  scoring='accuracy')

    print(model)
    print("Accuracy scores:\n", scores)
    print("Accuracy Average:\n", np.average(scores))
    print('\n')

In [ ]:
from sklearn.model_selection import cross_val_score

def CrossValidate_Model(model,X,y):
    # Multiply by -1 since sklearn calculates *negative* MAE
    scores = cross_val_score(model, X, y,
                                  cv=5,
                                  scoring='accuracy')

    print(model)
    print("Accuracy scores:\n", scores)
    print("Accuracy Average:\n", np.average(scores))
    print('\n')

In [ ]:
from sklearn.model_selection import GridSearchCV

def HyperParam_CV(model, X, y, param_grid):
        grid_search = GridSearchCV(model,
                                  param_grid,
                                  cv=5,
                                  scoring='accuracy',
                                  return_train_score=True)
        grid_search.fit(X, y)
        print(grid_search.best_params_)
        print(grid_search.best_score_)
        return grid_search.best_estimator_


Function to cross validate a given model by accuracy and to apply a Grid Search on a range of hyperparameters for a given model.
The function returns each distinct model with it's best performing combination of hyperparameters.

In [ ]:
from sklearn.model_selection import GridSearchCV

def HyperParam_CV(model, X, y, param_grid):
        grid_search = GridSearchCV(model,
                                  param_grid,
                                  cv=5,
                                  scoring='accuracy',
                                  return_train_score=True)
        grid_search.fit(X, y)
        print(grid_search.best_params_)
        print(grid_search.best_score_)
        return grid_search.best_estimator_


Evaluating 3 classification models, as well as number of hyperparameters with various potential settings. Here I am running a basic cross validation before running the CV with hyperparameter optimization.

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier

dtc = DecisionTreeClassifier(random_state=0)
dtc_param_grid = [{
    'min_samples_split': [1,2,5],
    'max_features': ['auto','sqrt']
}]

rfc = RandomForestClassifier(random_state=0)
rfc_param_grid = [{
    'n_estimators': [100,50,200],
    'max_features': [None,'auto','sqrt']
}]

xgb = XGBClassifier()
xgb_param_grid = [{
    "learning_rate":[0.3, 0.1, 0.03],
    "n_estimators":[100,50,200]
}]

models = [dtc,
          rfc,
          xgb]

param_grids = [dtc_param_grid, 
               rfc_param_grid, 
               xgb_param_grid]

for model in models:
    CrossValidate_Model(model,passengers_prepared,passengers_labels)

The RandomForestClassifier has the best average accuracy across the 5 folds in cross validation

In [ ]:
dtc_final_model = HyperParam_CV(models[0], passengers_prepared, passengers_labels, param_grids[0])    

{'max_features': None, 'min_samples_split': 10}

0.7302487243505287

In [ ]:
rfc_final_model = HyperParam_CV(models[1], passengers_prepared, passengers_labels, param_grids[1])  

{'bootstrap': True, 'max_features': 'auto', 'min_samples_leaf': 4, 'min_samples_split': 2, 'n_estimators': 100}

0.797657476784867

In [ ]:
xgb_final_model = HyperParam_CV(models[2], passengers_prepared, passengers_labels, param_grids[2])    

{'colsample_bytree': 1, 'learning_rate': 0.1, 'max_depth': 2, 'min_child_weight': 5, 'n_estimators': 100, 'subsample': 0.75}

0.795700014094843

# LazyPredict

When in Rome do as the Roman, well when Kaggling do as the Kagglers. Meaning do anything, no matter how impractical within data science's standard and norms, as long as it improves your overall score. On a more serious note, there's a lot of buzz around AutoML. After looking into its implementation with Python, I was pleasantly surprised to learn its actually a lot easier than I expected. Below is me using the library LazyPredict, which is an AutoML library. In this case I am use LazyPredict to only evaluate models a far wider range of model than I am proficient in.

In [ ]:
pip install lazypredict

Unfortunately, LazyPredict doesn't play well with CrossValidation, so I performed the standard train test split, but running the model is a cakewalk afterwards.

In [ ]:
from lazypredict.Supervised import LazyClassifier
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(passengers_prepared, 
                                                    passengers_labels, 
                                                    test_size=.33, 
                                                    random_state=42)

clf = LazyClassifier(predictions=True)
lazy_models, lazy_predictions = clf.fit(X_train, X_test, y_train, y_test)

lazy_models


In [ ]:
from lightgbm import LGBMClassifier

lgbm = LGBMClassifier(random_state=0)

lgbm_param_grid = [{
    "learning_rate":[0.3, 0.1, 0.03],
    "n_estimators":[100,50,200]
}]

lgbm_final_model = HyperParam_CV(lgbm, passengers_prepared, passengers_labels, lgbm_param_grid)    

# End of training
# Time to catch up the test set data predict on it

Now I need to bring my test set up to speed with my training set. Always satisfying after modulizing being able to do so in 2 line of code.

In [ ]:
testset_passengers = earlyTransformations(testset_passengers, is_training_set=False)
testset_passengers_prepared = full_pipeline.fit_transform(testset_passengers)

Now that the test set is up to speed with no errors, were ready to predict and submit.

In [ ]:
test_preds = rfc_final_model.predict(testset_passengers_prepared)
test_preds

In [ ]:
submission['Transported'] = test_preds.astype("bool")
submission.to_csv("submission.csv",index=False)
submission.head()